# Demo 1 - LLM Current Time
The issue here is that asking for last month's data doesn't always work since the LLM thinks it's 2023.

In [3]:
from sql_agent import initialize_db_connection, get_sql_database, initialize_llm, create_agent as create_sql_agent, run_query

engine, session = initialize_db_connection()
db = get_sql_database(engine)
llm = initialize_llm()
agent = create_sql_agent(db, llm)

Database connection established!
2025-04-13 08:20:48,315 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-04-13 08:20:48,315 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-13 08:20:48,319 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-04-13 08:20:48,319 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-13 08:20:48,321 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-04-13 08:20:48,321 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-13 08:20:48,323 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-13 08:20:48,324 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `finquery`
2025-04-13 08:20:48,325 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-13 08:20:48,344 INFO sqlalchemy.engine.Engine ROLLBACK
2025-04-13 08:20:48,348 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-13 08:20:48,349 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `finquery`
2025-04-13 08:20:48,351 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-13 08:20:48,364

In [4]:
test_query1 = "What were the top-selling products last month?"

## LLM date confusion
Note that a reasoning step says `First, I need to determine the date range for last month. Assuming the current date is in October 2023, last month would be September 2023.`

In [5]:
response = run_query(agent, test_query1)
print(response)

sql_agent.run_query: query='What were the top-selling products last month?'


> Entering new SQL Agent Executor chain...


/Users/ericmelz/Data/code/finquery/notebooks/sql_agent.py:83: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)


Action: sql_db_list_tables  
Action Input: ""  accounting_transactions, financial_transactions, sales_dataI need to check the schema of the `sales_data` table, as it is likely to contain information about products and their sales.  
Action: sql_db_schema  
Action Input: "sales_data"  2025-04-13 08:21:01,506 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-13 08:21:01,507 INFO sqlalchemy.engine.Engine SELECT sales_data.order_id, sales_data.product_id, sales_data.product_name, sales_data.category, sales_data.quantity, sales_data.unit_price, sales_data.total_price, sales_data.order_date 
FROM sales_data 
 LIMIT %(param_1)s
2025-04-13 08:21:01,507 INFO sqlalchemy.engine.Engine [generated in 0.00128s] {'param_1': 3}
2025-04-13 08:21:01,511 INFO sqlalchemy.engine.Engine ROLLBACK

CREATE TABLE sales_data (
	order_id VARCHAR(10), 
	product_id VARCHAR(10), 
	product_name VARCHAR(100), 
	category VARCHAR(50), 
	quantity INTEGER, 
	unit_price DECIMAL(10, 2), 
	total_price DECIMAL(10, 2), 
	